# Abstract

Make the test comparison associations

# Environment

In [1]:
# Make all associations
MAKE_ALL = False

In [2]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [3]:
from os import listdir, path
import pdb
import re
from tempfile import TemporaryDirectory

In [4]:
import numpy as np

In [5]:
from jwst.associations import (
    AssociationRegistry,
    generate
)
from jwst.associations.main import Main

In [6]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

# Library

In [31]:
def make_standards(pool_root, main_args=None):
    """Make the association standards for a pool"""
    pool_path = path.join(
        '..',
        'data',
        pool_root + '.csv'
    )
    pool = combine_pools(pool_path)
    if main_args is None:
        main_args = []
    with TemporaryDirectory() as tmp_path:
        args = main_args + ['-v', '-p', tmp_path]
        results = Main(
            args,
            pool=pool
        )
        asn_paths = listdir(tmp_path)
        for asn_path in asn_paths:
            matches = re.match('[^-]+(-.+)', asn_path)
            asn_tail = matches.group(1)
            from_path = path.join(tmp_path, asn_path)
            to_path = path.join('..', 'data', pool_root + asn_tail)
            %mv $from_path $to_path


# Main

## Level 2 Associations

In [8]:
lv2_only_args = [
    '-r',
    '../../lib/rules_level2b.py',
    '--ignore-default',
]

In [9]:
def_args = []

### pool_009_spec_miri_lv2bkg

In [21]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_009_spec_miri_lv2bkg', main_args=lv2_only_args)

### pool_010_spec_nirspec_lv2bkg

In [22]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_010_spec_nirspec_lv2bkg', main_args=lv2_only_args)

### pool_011_spec_miri_lv2bkg_lrs

In [23]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_011_spec_miri_lv2bkg_lrs', main_args=lv2_only_args)

### pool_013_coron_nircam

In [24]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_013_coron_nircam', main_args=def_args)

### pool_014_ami_niriss

In [25]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_014_ami_niriss', main_args=def_args)

### pool_015_spec_nirspec_lv2bkg_reversed

In [26]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_015_spec_nirspec_lv2bkg_reversed', main_args=lv2_only_args)

### pool_016_spec_nirspec_lv2bkg_double

In [27]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_016_spec_nirspec_lv2bkg_double', main_args=lv2_only_args)

### pool_017_spec_nirspec_lv2imprint

In [28]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_017_spec_nirspec_lv2imprint', main_args=lv2_only_args)

### pool_018_all_exptypes

In [29]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_018_all_exptypes', main_args=lv2_only_args)

### pool_019_niriss_wfss

In [33]:
make_me = MAKE_ALL
make_me = True
if make_me:
    make_standards('pool_019_niriss_wfss', main_args=def_args)

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpz7wnvzm4']
Reading rules.
Generating associations.


201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_lev

201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201709211107:WARNING:jwst.associations.lib.rules_lev

There where 4 associations and 2 orphaned items found.
Associations found are:
jw99009-o001_image3_001_asn with 1 products
Rule=candidate_Asn_Image
Constraints:
    wfsvisit: Is Invalid
    exp_type: nis_image
    opt_elem: f140m
    opt_elem2: None
    target: 1
    program: 99009
    instrument: niriss
    asn_candidate: \(\'o001\'\,\ \'observation\'\)
Products:
	jw99009-o001_t001_niriss_f140m with 4 members
jw99009-o001_spec3_001_asn with 1 products
Rule=candidate_Asn_WFSS
Constraints:
    exp_type: nis_wfss
    opt_elem2: gr150r|gr150c
    opt_elem: f140m
    target: 1
    program: 99009
    instrument: niriss
    asn_candidate: \(\'o001\'\,\ \'observation\'\)
Products:
	jw99009-o001_t001_niriss_f140m-gr150r|gr150c with 12 members
Association jw99009-o001_image2_001_asn
    Parameters:
        Product type: image2
        Rule:         candidate_Asn_Lv2Image
        Program:      99009
        Target:       1
        Pool:         pool_019_niriss_wfss
        Constraints:
         